<a href="https://colab.research.google.com/github/profteachkids/StemUnleashed/blob/main/ZoneSignal_DecisionTreeipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -q https://raw.githubusercontent.com/profteachkids/StemUnleashed/main/ZoneSignal.csv

In [2]:
import numpy as np
import pandas as pd

In [3]:
df=pd.read_csv('ZoneSignal.csv', index_col=0)

In [4]:
X=df.iloc[:,:-1].to_numpy()
Y=df.iloc[:,-1].to_numpy()

In [5]:
def splitGini(idxs, cols, X, Y, nCat):

    N=idxs.size
    bestGini=N
    for col in cols:
        sorted_idx = idxs[np.argsort(X[idxs,col])]
        countsL=np.zeros((N,nCat))
        countsL[range(N),Y[sorted_idx]]=1
        countsL=np.cumsum(countsL,axis=0)
        countsR=countsL[-1][None,:]-countsL

        #do not split after last item
        countsL=countsL[:-1]; countsR=countsR[:-1]
        probL=countsL/np.arange(1,N)[:,None]
        probR=countsR/(np.arange(N-1,0,-1)[:,None])

        giniL = np.arange(1,N)*(1-np.sum(probL**2,axis=1))
        giniR = np.arange(N-1,0,-1)*(1-np.sum(probR**2,axis=1))

        gini = giniL+giniR
        splitidx = np.argmin(gini)
        if gini[splitidx] < bestGini:
            bestGini= gini[splitidx]
            bestIdx=splitidx
            bestCol=col
            thresh = (X[sorted_idx[bestIdx],col]+X[sorted_idx[bestIdx+1],col])/2
            leftIdxs = sorted_idx[:bestIdx+1]
            rightIdxs = sorted_idx[bestIdx+1:]        
    return bestIdx, bestCol, thresh, leftIdxs, rightIdxs

In [9]:
def create_node(col, thresh, left_F=None, right_F=None):

    def setL(left):
        nonlocal left_F
        left_F = left

    def setR(right):
        nonlocal right_F
        right_F = right

    def node(x):
        if x[col]<thresh:
            if callable(left_F): 
                return left_F(x)
            else:
                return left_F
        else:
            if callable(right_F): 
                return right_F(x)
            else:
                return right_F
        
    return node, setL, setR

def createTree(stack, maxdepth, X, Y):

    nCat = Y.max()+1
    while stack:
        level, idxs, set_parent = stack.pop()
        vals, counts = np.unique(Y[idxs], return_counts=True)

        if len(counts)==1:
            set_parent(vals[0])
            continue
        if level==maxdepth:
            set_parent(np.argmax(counts))
            continue
        bestIdx, bestCol, thresh, leftIdxs, rightIdxs = splitGini(idxs, np.arange(0,X.shape[1]), X, Y, nCat)
        vals, counts = np.unique(Y[leftIdxs], return_counts=True)
        node, setL, setR = create_node(bestCol, thresh)
        stack.append((level+1, leftIdxs, setL))
        stack.append((level+1, rightIdxs, setR))
        if level == 0:
            rootNode = node
        else:
            set_parent(node)


    return rootNode

In [12]:
stack = [(0,np.arange(0,Y.size), None)]
tree=createTree(stack, 6, X, Y)

In [13]:
correct = 0
for x,y in zip(X,Y):
    rank=tree(x)
    if rank==y:
        correct+=1

print(f'fraction correct: {correct/Y.size}')

fraction correct: 0.940677966101695
